### Define file path

In [5]:
# 파일 경로 설정
import pandas as pd
import os

# PC filepath 설정
BASE_FILEPATH = "/home/songmu/"

# 사용할 client 및 server filepath 설정(연, 월, 일, 시)
CLIENT_FILEPATH = BASE_FILEPATH + "Multipath/cpp/results/client/2024_09_25_15_09/"
SERVER_FILEPATH = BASE_FILEPATH + "Multipath/cpp/results/server/2024_09_25_15_07/"

# 패킷 수신시 패킷 정보 logging 해둔 filepath
KT_LOG_FILEPATH = SERVER_FILEPATH + "logs/kt_log.csv"
LG_LOG_FILEPATH = SERVER_FILEPATH + "logs/lg_log.csv"
# 위 KT, LG 를 이용하여 만들어줄 Combine traffic의 저장 경로
COMBINE_LOG_FILEPATH = SERVER_FILEPATH + "logs/combine_log.csv"

# 송신측의 패킷 정보 logging 해둔 filepath
SENDING_LOG_FILEPATH = CLIENT_FILEPATH + "logs/packet_log.csv"

# 수신측에서 raw packet 저장한 filepath 
KT_BINS_FILEPATH = SERVER_FILEPATH + "bins/kt"
LG_BINS_FILEPATH = SERVER_FILEPATH + "bins/lg"
# 위 bins file과 Combine traffic logging을 이용하여 새로운 bin file을 저장해줄 filepath
COMBINE_BINS_FILEPATH = SERVER_FILEPATH + "bins/combine"
os.makedirs(COMBINE_BINS_FILEPATH, exist_ok = True)

# 생성한 그래프들 저장할 filepath
GRAPH_PATH = SERVER_FILEPATH + "graphs"
os.makedirs(GRAPH_PATH, exist_ok = True)

# 생성할 frames 저장할 filepath
KT_FRAMES_FILEPATH = SERVER_FILEPATH + "frames/kt"
LG_FRAMES_FILEPATH = SERVER_FILEPATH + "frames/lg"
COMBINE_FRAMES_FILEPATH = SERVER_FILEPATH + "frames/combine"
os.makedirs(KT_FRAMES_FILEPATH, exist_ok = True)
os.makedirs(LG_FRAMES_FILEPATH, exist_ok = True)
os.makedirs(COMBINE_FRAMES_FILEPATH, exist_ok = True)

### Make combine_log.csv

In [9]:
import pandas as pd

# Read csv files
kt_log_df = pd.read_csv(KT_LOG_FILEPATH, header=0, names=["source_ip", "sequence_number", "timestamp_frame", "timestamp_sending", "received_time", "network_latency_ms", "message_size"])
lg_log_df = pd.read_csv(LG_LOG_FILEPATH, header=0, names=["source_ip", "sequence_number", "timestamp_frame", "timestamp_sending", "received_time", "network_latency_ms", "message_size"])
sending_log_df = pd.read_csv(SENDING_LOG_FILEPATH)

# 두 개의 DataFrame을 sequence_number 기준으로 병합 (outer join, 양쪽 모두에 존재하는 행을 포함)
merged_df = pd.merge(kt_log_df, lg_log_df, on="sequence_number", how="outer", suffixes=('_kt', '_lg'))

# 빈 리스트 생성
combined_data = []

# 같은 sequence_number에서 latency가 적은 row를 선택하고, 마지막 column에 kt 또는 lg 추가
for index, row in merged_df.iterrows():
    if pd.isna(row['network_latency_ms_kt']):  # KT 데이터가 없을 경우 LG 데이터를 사용하고 'lg'를 추가
        combined_data.append([row['source_ip_lg'], row['sequence_number'], row['timestamp_frame_lg'], row['timestamp_sending_lg'], row['received_time_lg'], row['network_latency_ms_lg'], row['message_size_lg'], 'lg'])
    elif pd.isna(row['network_latency_ms_lg']):  # LG 데이터가 없을 경우 KT 데이터를 사용하고 'kt'를 추가
        combined_data.append([row['source_ip_kt'], row['sequence_number'], row['timestamp_frame_kt'], row['timestamp_sending_kt'], row['received_time_kt'], row['network_latency_ms_kt'], row['message_size_kt'], 'kt'])
    else:
        # 두 row 중 latency가 더 작은 것을 선택하고 그에 맞는 label 추가
        if row['network_latency_ms_kt'] < row['network_latency_ms_lg']:
            combined_data.append([row['source_ip_kt'], row['sequence_number'], row['timestamp_frame_kt'], row['timestamp_sending_kt'], row['received_time_kt'], row['network_latency_ms_kt'], row['message_size_kt'], 'kt'])
        else:
            combined_data.append([row['source_ip_lg'], row['sequence_number'], row['timestamp_frame_lg'], row['timestamp_sending_lg'], row['received_time_lg'], row['network_latency_ms_lg'], row['message_size_lg'], 'lg'])

# 결과를 DataFrame으로 변환
combined_log_df = pd.DataFrame(combined_data, columns= ["source_ip", "sequence_number", "timestamp_frame", "timestamp_sending", "received_time", "network_latency_ms", "message_size", "source"])
combined_log_df.to_csv(COMBINE_LOG_FILEPATH, index=False, header=True)

print(f"결과 파일이 {COMBINE_LOG_FILEPATH}에 저장되었습니다.")

결과 파일이 /home/songmu/Multipath/cpp/results/server/2024_09_24_20_22/logs/combine_log.csv에 저장되었습니다.


### Make combine_bin files

In [10]:
# Make combine bins

import os
import shutil

for index, row in combined_log_df.iterrows():
    sequence_number = row['sequence_number']
    source = row['source']
    
    # 복사할 파일 경로 설정
    if source == 'kt':
        source_folder = KT_BINS_FILEPATH
    elif source == 'lg':
        source_folder = LG_BINS_FILEPATH
    else:
        print(f"Unknown source: {source} at index {index}")
        continue
    
    # 파일명 찾기
    file_name = f"{sequence_number}_*.bins"
    source_file_path = os.path.join(source_folder, file_name)
    
    # 실제 파일명을 찾아 복사
    matching_files = [f for f in os.listdir(source_folder) if f.startswith(f"{sequence_number}_")]
    if not matching_files:
        print(f"No matching file found for sequence_number {sequence_number} in {source_folder}")
        continue
    
    # 첫 번째 매칭된 파일을 복사
    file_to_copy = matching_files[0]
    shutil.copy(os.path.join(source_folder, file_to_copy), os.path.join(COMBINE_BINS_FILEPATH, file_to_copy))
    
    # print(f"Copied {file_to_copy} to {combine_bins_folder}")
print('Copy completed')

Copy completed


### Generate frame (run c++ code)

In [11]:
# 현재는 오로지 network latency만 고려함

!bash run.sh

-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found PkgConfig: /usr/bin/pkg-config (found version "0.29.1") 
-- Checking for module 'libavcodec'
--   Found libavcodec, version 61.12.100
-- Checking for module 'libavutil'
--   Found libavutil, version 59.36.100
-- Checking for module 'libswscale'
--   Found libswscale, version 8.2.100
-- Checking for module 'opencv4'
--   Found opencv4, version 4.6.0
-- Configuring done
-- Generating done
-- Buil

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# 가장 높은 Sequence Number 찾기 (전체 패킷 수 계산)
max_sequence = sending_df['SequenceNumber'].max() + 1  # Sequence number는 0부터 시작하므로 +1
total_sequences = max_sequence

# 전체 수신된 패킷 비율 (잘 수신된 패킷 비율 계산)
kt_received = kt_df['sequence_number'].nunique()
lg_received = lg_df['sequence_number'].nunique()
combine_received = combined_df['sequence_number'].nunique()

kt_received_rate = (kt_received / total_sequences) * 100
lg_received_rate = (lg_received / total_sequences) * 100
combine_received_rate = (combine_received / total_sequences) * 100

# Latency 기준으로 수신된 패킷 비율 계산 (33ms, 50ms, 100ms 이상)
latency_thresholds = [33, 50, 100]
received_rate_dict = {}

for threshold in latency_thresholds:
    kt_received_latency = total_sequences - kt_df[kt_df['latency_ms'] > threshold]['sequence_number'].nunique()
    lg_received_latency = total_sequences - lg_df[lg_df['latency_ms'] > threshold]['sequence_number'].nunique()
    combine_received_latency = total_sequences - combined_df[combined_df['latency_ms'] > threshold]['sequence_number'].nunique()
    
    received_rate_dict[threshold] = {
        'kt': (kt_received_latency / total_sequences) * 100,
        'lg': (lg_received_latency / total_sequences) * 100,
        'combine': (combine_received_latency / total_sequences) * 100
    }

# 그래프 그리기
def plot_received_comparison(total_received_rate, total_sequences, received_rate_dict, latency_thresholds, graphs_path):
    if not os.path.exists(graphs_path):
        os.makedirs(graphs_path)
    
    # 전체 잘 수신된 패킷 비율 그래프
    fig, ax = plt.subplots(figsize=(10, 6))
    
    labels = ['KT', 'LG', 'Combine']
    received_rates = [total_received_rate['kt'], total_received_rate['lg'], total_received_rate['combine']]
    
    ax.bar(labels, received_rates, color=['blue', 'green', 'red'])
    
    # 각 바에 텍스트로 수신된 패킷 비율 출력
    for i, val in enumerate(received_rates):
        ax.text(i, val + 0.5, f'{val:.2f}%\n({int(val/100 * total_sequences)} / {total_sequences})', ha='center', va='bottom')
    
    ax.set_title(f"Overall Packet Reception Rate (Total Sequences: {total_sequences})")
    ax.set_xlabel("Mode")
    ax.set_ylabel("Packet Reception Rate (%)")
    
    overall_graph_path = os.path.join(graphs_path, "overall_packet_reception_rate.png")
    plt.savefig(overall_graph_path)
    plt.close()

    # Latency threshold 별로 수신된 패킷 비율 비교
    for threshold in latency_thresholds:
        fig, ax = plt.subplots(figsize=(10, 6))
        
        latency_received_rates = [
            received_rate_dict[threshold]['kt'],
            received_rate_dict[threshold]['lg'],
            received_rate_dict[threshold]['combine']
        ]
        
        ax.bar(labels, latency_received_rates, color=['blue', 'green', 'red'])
        
        for i, val in enumerate(latency_received_rates):
            ax.text(i, val + 0.5, f'{val:.2f}%\n({int(val/100 * total_sequences)} / {total_sequences})', ha='center', va='bottom')
        
        ax.set_title(f"Packet Reception Rate with Latency < {threshold} ms (Total Sequences: {total_sequences})")
        ax.set_xlabel("Mode")
        ax.set_ylabel(f"Packet Reception Rate (%) with Latency < {threshold}ms")
        
        latency_graph_path = os.path.join(graphs_path, f"packet_reception_rate_latency_{threshold}ms.png")
        plt.savefig(latency_graph_path)
        plt.close()

# 총 잘 수신된 패킷 비율 딕셔너리 생성
total_received_rate = {
    'kt': kt_received_rate,
    'lg': lg_received_rate,
    'combine': combine_received_rate
}

# 그래프 그리기 및 저장
plot_received_comparison(total_received_rate, total_sequences, received_rate_dict, latency_thresholds, graphs_path)


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# 전체 평균 Latency 계산
kt_avg_latency = kt_df['latency_ms'].mean()
lg_avg_latency = lg_df['latency_ms'].mean()
combine_avg_latency = combined_df['latency_ms'].mean()

# Latency 기준으로 평균 Latency 계산 (33ms, 50ms, 100ms 이상은 제외)
latency_thresholds = [33, 50, 100]
avg_latency_dict = {}

for threshold in latency_thresholds:
    kt_filtered = kt_df[kt_df['latency_ms'] <= threshold]
    lg_filtered = lg_df[lg_df['latency_ms'] <= threshold]
    combine_filtered = combined_df[combined_df['latency_ms'] <= threshold]
    
    avg_latency_dict[threshold] = {
        'kt': kt_filtered['latency_ms'].mean() if not kt_filtered.empty else None,
        'lg': lg_filtered['latency_ms'].mean() if not lg_filtered.empty else None,
        'combine': combine_filtered['latency_ms'].mean() if not combine_filtered.empty else None
    }

# 그래프 그리기
def plot_latency_comparison(total_avg_latency, avg_latency_dict, latency_thresholds, graphs_path):
    if not os.path.exists(graphs_path):
        os.makedirs(graphs_path)
    
    # 전체 평균 Latency 그래프
    fig, ax = plt.subplots(figsize=(10, 6))
    
    labels = ['KT', 'LG', 'Combine']
    avg_latencies = [total_avg_latency['kt'], total_avg_latency['lg'], total_avg_latency['combine']]
    
    ax.bar(labels, avg_latencies, color=['blue', 'green', 'red'])
    
    # 각 바에 텍스트로 평균 Latency 출력
    for i, val in enumerate(avg_latencies):
        ax.text(i, val + 0.5, f'{val:.2f} ms', ha='center', va='bottom')
    
    ax.set_title("Overall Average Latency Comparison")
    ax.set_xlabel("Mode")
    ax.set_ylabel("Average Latency (ms)")
    
    overall_latency_graph_path = os.path.join(graphs_path, "overall_average_latency_comparison.png")
    plt.savefig(overall_latency_graph_path)
    plt.close()

    # Latency threshold 별로 평균 Latency 비교
    for threshold in latency_thresholds:
        fig, ax = plt.subplots(figsize=(10, 6))
        
        latency_avg_latencies = [
            avg_latency_dict[threshold]['kt'],
            avg_latency_dict[threshold]['lg'],
            avg_latency_dict[threshold]['combine']
        ]
        
        ax.bar(labels, latency_avg_latencies, color=['blue', 'green', 'red'])
        
        for i, val in enumerate(latency_avg_latencies):
            if val is not None:
                ax.text(i, val + 0.5, f'{val:.2f} ms', ha='center', va='bottom')
            else:
                ax.text(i, 0.5, 'N/A', ha='center', va='bottom')  # 데이터가 없는 경우 'N/A' 표시
        
        ax.set_title(f"Average Latency Comparison with Latency <= {threshold} ms")
        ax.set_xlabel("Mode")
        ax.set_ylabel(f"Average Latency (ms) with Latency <= {threshold}ms")
        
        latency_graph_path = os.path.join(graphs_path, f"average_latency_comparison_latency_{threshold}ms.png")
        plt.savefig(latency_graph_path)
        plt.close()

# 총 평균 Latency 딕셔너리 생성
total_avg_latency = {
    'kt': kt_avg_latency,
    'lg': lg_avg_latency,
    'combine': combine_avg_latency
}

# 그래프 그리기 및 저장
plot_latency_comparison(total_avg_latency, avg_latency_dict, latency_thresholds, graphs_path)


In [64]:
import os
import shutil

# 원본 frames 폴더 경로
frames_folder = os.path.expanduser("/home/songmu/Downloads/client/frames")
# 새로 생성할 frames_with_sequence 폴더 경로
destination_folder = os.path.expanduser("/home/songmu/Downloads/client/frames_with_sequence")

# frames_with_sequence 폴더가 없으면 생성
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# frames 폴더 내의 파일들을 가져옴
files = [f for f in os.listdir(frames_folder) if f.endswith(".png")]

# 파일들을 타임스탬프 기준으로 정렬
files.sort(key=lambda x: int(x.split('_')[1].split('.')[0]))

# 시퀀스 넘버를 2부터 시작
sequence_number = 2

# 파일들을 순서대로 복사하며 이름 변경
for file_name in files:
    timestamp = file_name.split('_')[1].split('.')[0]
    new_file_name = f"{sequence_number}_{timestamp}.png"
    src_path = os.path.join(frames_folder, file_name)
    dst_path = os.path.join(destination_folder, new_file_name)
    shutil.copy(src_path, dst_path)
    sequence_number += 1

print("Files have been successfully copied and renamed.")


Files have been successfully copied and renamed.


In [75]:
import os
import cv2
from skimage.metrics import structural_similarity as ssim

# 경로 설정
original_folder = "/home/songmu/Downloads/client/frames_with_sequence"
kt_folder = "/home/songmu/Downloads/frames/kt"
lg_folder = "/home/songmu/Downloads/frames/lg"
combine_folder = "/home/songmu/Downloads/frames/combine"

# 결과를 저장할 딕셔너리 초기화
results = {
    "kt": {"ssim": [], "psnr": []},
    "lg": {"ssim": [], "psnr": []},
    "combine": {"ssim": [], "psnr": []}
}

# 두 이미지를 비교하여 SSIM과 PSNR을 반환하는 함수
def compare_images(img1, img2):
    # 이미지의 작은 쪽 크기를 기준으로 win_size를 설정
    min_side = min(img1.shape[:2])
    win_size = min(7, min_side)  # win_size는 7이거나 이미지의 작은 쪽 크기 이하여야 함
    ssim_value = ssim(img1, img2, win_size=win_size, channel_axis=-1)
    psnr_value = cv2.PSNR(img1, img2)
    return ssim_value, psnr_value

# 원본 이미지 파일 리스트 가져오기 및 정렬
original_files = [f for f in os.listdir(original_folder) if f.endswith(".png")]
original_files.sort(key=lambda x: int(x.split('_')[0]))

# 원본 이미지 파일을 순회하며 비교
for original_file in original_files:
    sequence_number = original_file.split('_')[0]

    # 원본 이미지를 불러오기
    original_img = cv2.imread(os.path.join(original_folder, original_file))

    # 비교할 대상 폴더에서 같은 sequence_number를 가진 파일을 찾기
    for folder_name, folder_path in [("kt", kt_folder), ("lg", lg_folder), ("combine", combine_folder)]:
        compare_file = next((f for f in os.listdir(folder_path) if f.startswith(sequence_number)), None)
        
        if compare_file:
            compare_img = cv2.imread(os.path.join(folder_path, compare_file))
            
            # SSIM, PSNR 계산
            ssim_value, psnr_value = compare_images(original_img, compare_img)
            
            # 결과 저장
            results[folder_name]["ssim"].append(ssim_value)
            results[folder_name]["psnr"].append(psnr_value)
        else:
            print(f"Sequence {sequence_number} not found in {folder_name} folder, skipping comparison.")

# 결과 출력
for key in results:
    ssim_values = results[key]["ssim"]
    psnr_values = results[key]["psnr"]
    
    print(f"Results for {key}:")
    print(f"Average SSIM: {sum(ssim_values)/len(ssim_values):.4f}")
    print(f"Average PSNR: {sum(psnr_values)/len(psnr_values):.2f} dB")
    print("\n")


Sequence 113 not found in kt folder, skipping comparison.
Sequence 133 not found in lg folder, skipping comparison.
Sequence 134 not found in lg folder, skipping comparison.
Sequence 150 not found in kt folder, skipping comparison.
Sequence 178 not found in kt folder, skipping comparison.
Sequence 184 not found in kt folder, skipping comparison.
Sequence 186 not found in kt folder, skipping comparison.
Sequence 203 not found in kt folder, skipping comparison.
Sequence 204 not found in kt folder, skipping comparison.
Sequence 205 not found in kt folder, skipping comparison.
Sequence 254 not found in kt folder, skipping comparison.
Sequence 335 not found in kt folder, skipping comparison.
Sequence 336 not found in kt folder, skipping comparison.
Sequence 400 not found in lg folder, skipping comparison.
Sequence 421 not found in kt folder, skipping comparison.
Sequence 424 not found in kt folder, skipping comparison.
Sequence 445 not found in kt folder, skipping comparison.
Sequence 509 n

In [74]:
import os

# 경로 설정
original_folder = "/home/songmu/Downloads/client/frames_with_sequence"
kt_folder = "/home/songmu/Downloads/frames/kt"
lg_folder = "/home/songmu/Downloads/frames/lg"
combine_folder = "/home/songmu/Downloads/frames/combine"

# 폴더 경로 리스트
folders = {
    "original": original_folder,
    "kt": kt_folder,
    "lg": lg_folder,
    "combine": combine_folder
}

# FPS 계산 함수
def calculate_fps(folder_path, first_seq, last_seq):
    # 특정 시퀀스 번호 범위에 해당하는 파일 필터링
    frame_files = [f for f in os.listdir(folder_path) if f.endswith(".png")]
    
    # 시퀀스 번호와 타임스탬프 추출
    frame_info = [
        (int(f.split('_')[0]), int(f.split('_')[1].split('.')[0]))
        for f in frame_files
    ]
    
    # 주어진 시퀀스 번호 범위 내에서 첫 번째와 마지막 프레임 찾기
    first_frame = min([info for info in frame_info if first_seq <= info[0] <= last_seq], key=lambda x: x[0])
    last_frame = max([info for info in frame_info if first_seq <= info[0] <= last_seq], key=lambda x: x[0])
    
    # 전체 프레임 수
    total_frames = len([info for info in frame_info if first_seq <= info[0] <= last_seq])
    
    # 총 재생 시간 (마지막 타임스탬프 - 첫 번째 타임스탬프)
    total_playtime = (last_frame[1] - first_frame[1]) / 1000.0  # 밀리초(ms)에서 초(s)로 변환
    
    if total_playtime == 0:
        return 0
    
    # FPS 계산
    fps = total_frames / total_playtime
    return fps

# Original 폴더 FPS 계산
original_fps = calculate_fps(original_folder, 2, 869)
print(f"FPS for original: {original_fps:.2f}")

# 각 폴더의 FPS 계산 및 출력
for folder_name, folder_path in {"kt": kt_folder, "lg": lg_folder, "combine": combine_folder}.items():
    fps = calculate_fps(folder_path, 2, 853)
    print(f"FPS for {folder_name}: {fps:.8f}")


FPS for original: 0.03
FPS for kt: 0.02885453
FPS for lg: 0.02965896
FPS for combine: 0.02976388
